<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Advertools - Analyze website content using XML sitemap

**Tags:** #advertools #xml #sitemap #website #analyze #seo

**Author:** [Elias Dabbas](https://www.linkedin.com/in/eliasdabbas/)

**Description:** This notebook helps you get an overview of a website's content by analyzing and visualizing its XML sitemap. It's also an important SEO audit process that can uncover some potential issues that might affect the website.

**References:**
- [Advertools](https://github.com/eliasdabbas/advertools)
- [XML Sitemap](https://en.wikipedia.org/wiki/Sitemaps)

## Input

### Import libraries

In [ ]:
import advertools as adv

### Setup Variables
- `url`: URL of the website to analyze

In [ ]:
url = "https://www.example.com"

## Model

### Analyze website content using XML sitemap

This function will analyze the website content by extracting the XML sitemap and visualizing it.

In [ ]:
adv.sitemap_analyze(url)

## Output

### Display result

In [ ]:
adv.sitemap_visualize(url)